# Models for Age,Gender and Liveness
### for this partcular problem we are using fine tuned mobilenetv3 small model ,since mobile net models are smaller and faster and lighter than other models it will be computationaly efficient 
### mobilenet models can cause slight accuracy issue where it lacks in performace gains in efficiency


In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf 
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input,decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout,BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import SparseCategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import matplotlib.pyplot as plt 
import cv2

2025-05-10 06:52:52.663902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-10 06:52:52.672684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746840172.683055   28924 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746840172.686236   28924 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-10 06:52:52.697641: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### preprocessing

In [2]:
# img = image.load_img("image.jpeg")
# img = image.img_to_array(img)
# img = np.expand_dims(img,axis=0)
# img = preprocess_input(img)

In [3]:
base_model = MobileNetV3Small(input_shape=(224, 224, 3),weights="imagenet",include_top=False)

2025-05-10 06:52:54.793202: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-05-10 06:52:54.793228: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: rijo-Thin-15-B12UCX
2025-05-10 06:52:54.793232: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: rijo-Thin-15-B12UCX
2025-05-10 06:52:54.793357: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.144.3
2025-05-10 06:52:54.793373: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.144.3
2025-05-10 06:52:54.793375: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.144.3


In [4]:
base_model.summary()

Model: "MobileNetV3Small"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ rescaling[0][0]   │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 113, 113,  │          0 │ activation[0][0]  │
│ (ZeroPadding2D)     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 56, 56,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │          0 │ re_lu[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │        136 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │          0 │ expanded_conv_sq… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │        144 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 1, 16)  │          0 │ expanded_conv_sq… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 1, 1, 16)  │          0 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 1, 1, 16)  │          0 │ re_lu_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 56, 56,    │          0 │ re_lu[0][0],      │
│ (Multiply)          │ 16)               │            │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 56, 56,    │        256 │ expanded_conv_sq

 Total params: 939,120 (3.58 MB)

 Trainable params: 927,008 (3.54 MB)

 Non-trainable params: 12,112 (47.31 KB)

In [5]:
len(base_model.layers[:-1])

156

## Base model function

In [6]:
def adaptive_model_transfer_learning_function(base_model,no_of_classes):
    for layer in base_model.layers:
        layer.trainable=False 

    # create new output layer
    x=GlobalAveragePooling2D()(base_model.output)
    x=Dense(256,activation="relu")(x)
    x=BatchNormalization()(x)
    x=Dropout(0.3)(x)
    x=Dense(128,activation="relu")(x)
    x=Dropout(0.2)(x)
    output=Dense(no_of_classes)(x)

    # build final model
    model = Model(inputs=base_model.input, outputs=output)
    return model

             

### Gender detection Model 

In [7]:
train_datagen = ImageDataGenerator(
    zoom_range=0.2,
    rotation_range=0.2, 
    brightness_range=(0.9,1.1),
   
    preprocessing_function=preprocess_input
    
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [8]:
train_data = train_datagen.flow_from_directory(
                        "GENDER_DATASET/Training/",
                         target_size=(224, 224), 
                         class_mode="binary",
                         batch_size=32,
                         classes=["female","male"]
                    )

test_data = test_datagen.flow_from_directory(
       "GENDER_DATASET/Validation/", 
        target_size=(224, 224), 
        class_mode="binary",
        batch_size=32,
      
)

Found 47009 images belonging to 2 classes.
Found 11649 images belonging to 2 classes.


In [9]:
x, y = next(train_data)
print("Labels sample:", y[:20])
print("Unique labels:", np.unique(y))


Labels sample: [0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0.]
Unique labels: [0. 1.]


In [10]:
train_data.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [11]:
base_model_gender = MobileNetV3Small(input_shape=(224, 224, 3),weights="imagenet",include_top=False)
GENDER_MODEL = adaptive_model_transfer_learning_function(base_model_gender,1)
GENDER_MODEL.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss=BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [12]:
callback=EarlyStopping(patience=3,restore_best_weights=True)

In [13]:
GENDER_MODEL.fit(
    train_data,
    validation_data=test_data,
    epochs=5,
    batch_size=32,callbacks=[callback])

/home/rijo/.local/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 375s 253ms/step - accuracy: 0.8102 - loss: 0.4268 - val_accuracy: 0.9179 - val_loss: 0.2005
Epoch 2/5
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 368s 250ms/step - accuracy: 0.8960 - loss: 0.2508 - val_accuracy: 0.9169 - val_loss: 0.1883
Epoch 3/5
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 372s 253ms/step - accuracy: 0.9057 - loss: 0.2276 - val_accuracy: 0.9289 - val_loss: 0.1724
Epoch 4/5
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 374s 254ms/step - accuracy: 0.9105 - loss: 0.2186 - val_accuracy: 0.9260 - val_loss: 0.1713
Epoch 5/5
1470/1470 ━━━━━━━━━━━━━━━━━━━━ 373s 254ms/step - accuracy: 0.9140 - loss: 0.2111 - val_accuracy: 0.9372 - val_loss: 0.1657


In [14]:
GENDER_MODEL.evaluate(test_data)

365/365 ━━━━━━━━━━━━━━━━━━━━ 40s 110ms/step - accuracy: 0.9415 - loss: 0.1621


[0.1657363474369049, 0.9372478127479553]

In [15]:
GENDER_MODEL.save('GENDER.keras') 

## Liveness detection Model

In [16]:
train_datagen_liveness = ImageDataGenerator(
    zoom_range=0.2,
    rotation_range=0.2,
    brightness_range=(0.9, 1.1),
    validation_split=0.2 , 
    preprocessing_function=preprocess_input
)

train_data_liveness = train_datagen_liveness.flow_from_directory(
    "LIVENESS_DATASET",
    target_size=(224, 224),
    class_mode="binary",
    batch_size=32,
    classes=["Live", "Spoof"],
    subset="training"  
)


validation_data_liveness = train_datagen_liveness.flow_from_directory(
    "LIVENESS_DATASET",
    target_size=(224, 224),
    class_mode="binary",
    batch_size=32,
    classes=["Live", "Spoof"],
    subset="validation"  
)

Found 480 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


In [17]:
base_model_live = MobileNetV3Small(input_shape=(224, 224, 3),weights="imagenet",include_top=False)
LIVE_MODEL = adaptive_model_transfer_learning_function(base_model_live,1)
LIVE_MODEL.compile(
    optimizer=Adam(),
    loss=BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [18]:
LIVE_MODEL.fit(
    train_data_liveness,
    validation_data=validation_data_liveness,
    epochs=5,
    batch_size=32,callbacks=[callback])

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 393ms/step - accuracy: 0.6779 - loss: 0.5834 - val_accuracy: 0.8667 - val_loss: 0.3692
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 313ms/step - accuracy: 0.9724 - loss: 0.1057 - val_accuracy: 0.9500 - val_loss: 0.1926
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 313ms/step - accuracy: 0.9788 - loss: 0.0653 - val_accuracy: 0.9750 - val_loss: 0.1415
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 322ms/step - accuracy: 0.9826 - loss: 0.0523 - val_accuracy: 0.9500 - val_loss: 0.1368
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 314ms/step - accuracy: 0.9920 - loss: 0.0325 - val_accuracy: 0.9583 - val_loss: 0.1381


In [19]:
LIVE_MODEL.evaluate(validation_data_liveness)

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.9794 - loss: 0.1106


[0.12329867482185364, 0.9666666388511658]

In [20]:
LIVE_MODEL.save('LIVE.keras') 

In [21]:
# import os
# import shutil
# import pandas as pd


# folder_path = 'AGE_DATASET/Train' 
# csv_file = 'AGE_DATASET/train.csv' 

# df = pd.read_csv(csv_file)

# classes = df['Class'].unique()
# for cls in classes:
#     class_folder = os.path.join(folder_path, cls)
#     if not os.path.exists(class_folder):
#         os.makedirs(class_folder)

# for _, row in df.iterrows():
#     image_name = row['ID']
#     image_class = row['Class']
    
#     source_path = os.path.join(folder_path, image_name)
#     destination_path = os.path.join(folder_path, image_class, image_name)
    
#     if os.path.exists(source_path):
#         shutil.move(source_path, destination_path)
#     else:
#         print(f"Image {image_name} not found!")

# print("images have been successfully moved!")


## Age detection model

In [22]:
train_datagen_age = ImageDataGenerator(
    zoom_range=0.2,
    rotation_range=0.2,
    brightness_range=(0.9, 1.1),
    validation_split=0.2 ,
    preprocessing_function=preprocess_input
)

train_data_age = train_datagen_age.flow_from_directory(
    "AGE_DATASET/Train/",
    target_size=(224, 224),
    class_mode="sparse",
    batch_size=32,
    classes=["MIDDLE","OLD","YOUNG"],
    subset="training"  
)


validation_data_age = train_datagen_age.flow_from_directory(
    "AGE_DATASET/Train/",
    target_size=(224, 224),
    class_mode="sparse",
    batch_size=32,
    classes=["MIDDLE","OLD","YOUNG"],
    subset="validation"  
)

Found 15926 images belonging to 3 classes.
Found 3980 images belonging to 3 classes.


In [23]:
base_model_age = MobileNetV3Small(input_shape=(224, 224, 3),weights="imagenet",include_top=False)
AGE_MODEL = adaptive_model_transfer_learning_function(base_model_age,3)
AGE_MODEL.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [24]:
AGE_MODEL.fit(train_data_age,callbacks=[callback],epochs=5)

Epoch 1/5
498/498 ━━━━━━━━━━━━━━━━━━━━ 123s 242ms/step - accuracy: 0.5959 - loss: 0.9500
Epoch 2/5


/home/rijo/.local/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


498/498 ━━━━━━━━━━━━━━━━━━━━ 114s 229ms/step - accuracy: 0.6791 - loss: 0.7474
Epoch 3/5
498/498 ━━━━━━━━━━━━━━━━━━━━ 114s 229ms/step - accuracy: 0.6915 - loss: 0.7013
Epoch 4/5
498/498 ━━━━━━━━━━━━━━━━━━━━ 114s 228ms/step - accuracy: 0.7130 - loss: 0.6562
Epoch 5/5
498/498 ━━━━━━━━━━━━━━━━━━━━ 114s 229ms/step - accuracy: 0.7137 - loss: 0.6598


In [25]:
AGE_MODEL.save('AGE.keras') 